# Grants Sankey

In [1]:
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import sys
sys.path.append(os.path.abspath("../../../visualizations/"))
from sankey import make_sankey_graph

# Ingest the data

In [4]:
url = "https://raw.githubusercontent.com/opensource-observer/oss-funding/main/funding_data.csv"
df = pd.read_csv(url, index_col=0)
df['funding_usd_log'] = np.log10(df['funding_usd'])
df['funding_event_count'] = 1
df.sort_values(by='funding_usd').iloc[-1]

oso_slug                                                          gmx-io
project_name                                                         GMX
project_id                                                           NaN
project_url            https://forum.arbitrum.foundation/c/dao-grant-...
project_address                                                      NaN
funder_name                                          Arbitrum Foundation
funder_round_name                                                 STIP-1
funder_round_type                                                 growth
funder_address                                                       NaN
funding_amount                                                12000000.0
funding_currency                                                     ARB
funding_network                                                 arbitrum
funding_date                                                  2024-01-01
funding_usd                                        

# Filter on OSS projects that are repeat players

In [15]:
dff = (
    df[
        (df['oso_slug'].isna()==False) 
        & (df['oso_slug'].str.contains(',') == False)
    ]
)

repeat_projects = dff.groupby('oso_slug')['funding_event_count'].sum()
repeat_projects = list(repeat_projects[repeat_projects>1].index)

dff = dff[dff['oso_slug'].isin(repeat_projects)]
len(dff)/len(df)

0.20838323353293414

# Ecosystem Sankey variants

In [16]:
fig = make_sankey_graph(
    dff,
    cat_cols=['funder_name', 'funder_round_name', 'oso_slug'],
    value_col='funding_usd',
    title='Ethereum grant funding (WIP)',
    height=4000,
    main_color='purple',
    med_color='teal',
    light_color='green',
#    hide_label_cols=['oso_slug']
)
go.Figure(fig)
#go.Figure(fig).write_html("EthereumGrantFunding.html")

In [17]:
fig = make_sankey_graph(
    dff,
    cat_cols=['funder_name', 'funder_round_name', 'oso_slug'],
    value_col='funding_usd_log',
    title='Ethereum grant funding (WIP) | log scale',
    height=4000,
    main_color='purple',
    med_color='teal',
    light_color='green',
    #hide_label_cols=['oso_slug']
)
go.Figure(fig)

In [18]:
fig = make_sankey_graph(
    dff,
    cat_cols=['funder_name', 'funder_round_name', 'oso_slug'],
    value_col='funding_event_count',
    title='Ethereum grant funding | # of grants received by a project',
    height=4000,
    main_color='purple',
    med_color='teal',
    light_color='green',
    #hide_label_cols=['oso_slug']
)
go.Figure(fig)

# Make project level Sankeys

In [28]:
dff.groupby("oso_slug")['funding_usd'].sum().sort_values(ascending=False).head(20)

oso_slug
mux-world           1.087648e+07
synthetix           9.560658e+06
velodrome           7.627827e+06
synapse             4.585253e+06
stargate-finance    4.500000e+06
lyra-finance        3.715327e+06
pendle-finance      3.696000e+06
protocol-guild      3.387973e+06
kyber-swap          3.000000e+06
socket              2.642559e+06
thales              2.525000e+06
rabbithole          2.390309e+06
dforce-network      2.350000e+06
go-ethereum         2.107999e+06
defi-llama          2.075434e+06
premiafinance       1.927500e+06
ethers-io           1.853344e+06
solidity            1.804341e+06
rotki               1.661669e+06
lighthouse-sigp     1.653971e+06
Name: funding_usd, dtype: float64

In [30]:
slug = 'defi-llama'
dfff = dff[dff['oso_slug'] == slug]
fig = make_sankey_graph(
    dfff,
    cat_cols=['funder_name', 'funder_round_name', 'project_name'],
    value_col='funding_usd',
    title=f'Ethereum grant funding to {slug} | ${dfff["funding_usd"].sum():,.0f}',
    height=1000,
    main_color='purple',
    med_color='teal',
    light_color='green'
)
go.Figure(fig)